## 1. Get dataset (Business & data understanding)

>가장 먼저, pdf에서 제공받은 홈페이지에서 dataset을 가져와 읽어온다.

나는 포르투갈에서 진행한 은행 캠페인의 고객 dataset을 받아왔다.  
해당 dataset에서 각 열의 의미는 아래와 같다.  
- age : 고객의 나이
- job : 고객의 직업
- marital : 기혼 여부
- education : 교육 여부 (unknown, primary, secondary, teritary)
- default : 카드 요금 체납 여부
- balance : 연 평균 잔고 (유로 단위)
- housing : 주택 대출 여부
- loan : 개인 대출 여부
- contact : 연락 수신 유형 (unknown, telephone, cellular)
- day : 해당 달의 마지막 연락(contact) 일자
- month : 해당 연도의 마지막 연락(contact) 달
- duration : 마지막 contact 기간 (초 단위)
- campaign : 해당 campaign이 진행되는 동안, 고객에게 간 연락(contact)의 횟수
- pdays : 이전 campaign의 연락을 받은 뒤로 며칠이 지났는지?
  - -1은, 이전에 연락이 간 적이 없음을 의미한다.
- previous : 해당 campaign이 진행되기 전 고객에게 간 연락(contact)의 횟수
- poutcome : 이전 campaign에 대한 결과 (unknwon, other, failure, success)
- y : 위의 입력에 대한 결과 값이다.
  - 위의 입력에 대해 고객이 Term deposit(정기 예금)에 가입했는지 판단한다.




In [ ]:
import pandas as pd

col_names = ["age", "job", "marital", "education", "default", "balance", 
             "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome", "y"]

with open('/content/bank-full.csv', 'r') as file: 
  bank_data = pd.read_csv(file, sep = ";")

bank_data.head(15)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


## 2. Data preparation
> Data를 알고리즘에 따라 학습시키기 이전에, 전처리가 필요하다.

- Categorical data
  - 수치적 계산을 해야하기 때문에, categorical data로는 학습할 수 없다.
  - 따라서 factorize 함수를 통해 모두 수치값으로 변경하도록 한다.

In [ ]:
## Categorical data to Numeric data
bank_data['job'], _ = bank_data['job'].factorize()
bank_data['marital'], _ = bank_data['marital'].factorize()
bank_data['education'], _ = bank_data['education'].factorize()
bank_data['default'], _ = bank_data['default'].factorize()
bank_data['housing'], _ = bank_data['housing'].factorize()
bank_data['loan'], _ = bank_data['loan'].factorize()
bank_data['contact'], _ = bank_data['contact'].factorize()
bank_data['month'], _ = bank_data['month'].factorize()
bank_data['poutcome'], _ = bank_data['poutcome'].factorize()
bank_data['y'], _ = bank_data['y'].factorize()

## 3. Classification with decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

clf = DecisionTreeClassifier()
train_data = bank_data[col_names[0:16]]
train_label = bank_data['y']

## Train set만 두고 예측을 한번 진행해봄
clf.fit(train_data, train_label)

pred = clf.predict(train_data)
accuracy_score(train_label, pred)

## K-fold cross validation을 위해 Train / Test set 분리
x_train, x_test, y_train, y_test = train_test_split(train_data, train_label, test_size = 0.2, random_state=156)

# train data로 예측
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
accuracy_score(y_test, pred)

0.8781377861329205

In [ ]:
from sklearn.model_selection import KFold
# K-fold cross validation 진행
clf = DecisionTreeClassifier(random_state=156)
k_fold = KFold(n_splits=5)
iter = 0
accuracies = []

feature = bank_data[col_names[0:16]]
label = bank_data['y']

for train_index, test_index in k_fold.split(feature):
    x_train, x_test = feature.iloc[train_index], feature.iloc[test_index]
    y_train, y_test = label[train_index], label[test_index]

    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)
    iter += 1

    accuracy = np.round(accuracy_score(y_test, pred), 4) # 소수점 4자리 반올림
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]

    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(iter,test_index))
    accuracies.append(accuracy)
    
print(accuracies)


#1 교차 검증 정확도 : 0.8679,  학습 데이터 크기 : 36168,  검증 데이터 크기 : 9043
#1 검증 세트 인덱스 : [   0    1    2 ... 9040 9041 9042]

#2 교차 검증 정확도 : 0.7915,  학습 데이터 크기 : 36169,  검증 데이터 크기 : 9042
#2 검증 세트 인덱스 : [ 9043  9044  9045 ... 18082 18083 18084]

#3 교차 검증 정확도 : 0.7516,  학습 데이터 크기 : 36169,  검증 데이터 크기 : 9042
#3 검증 세트 인덱스 : [18085 18086 18087 ... 27124 27125 27126]

#4 교차 검증 정확도 : 0.6541,  학습 데이터 크기 : 36169,  검증 데이터 크기 : 9042
#4 검증 세트 인덱스 : [27127 27128 27129 ... 36166 36167 36168]

#5 교차 검증 정확도 : 0.7069,  학습 데이터 크기 : 36169,  검증 데이터 크기 : 9042
#5 검증 세트 인덱스 : [36169 36170 36171 ... 45208 45209 45210]
[0.8679, 0.7915, 0.7516, 0.6541, 0.7069]
